In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - remove_gcs.ipynb
# AUTHOR - James Lane
# PROJECT -
# CONTENTS:
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Find the fields with globular cluster stars in them that need to be removed 
from analysis
'''

__author__ = "James Lane"

In [ ]:
### Imports

# General
import numpy as np, pdb, sys, os, dill as pickle, copy
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import matplotlib

# Set APOGEE version for the package
apogee_results_vers = 'l33'
# Year 7 is appropriate for DR16 (l33)
apo_combined_select_year = 7
os.environ['RESULTS_VERS'] = apogee_results_vers

# Astronomy
import apogee.select as apsel
import apogee.tools as apotools
from isodist import Z2FEH
import mwdust
from astropy import units as apu
from astropy.io import fits
from astropy.coordinates import SkyCoord, Galactic

from galpy import orbit
from galpy import potential
from galpy import actionAngle as aA
from galpy.util import coords

# Project specific
sys.path.insert(0,'../../src/')
from ges_mass import iso as piso
from ges_mass import util as putil
from ges_mass import ssf as pssf
from ges_mass import plot as pplot

In [ ]:
# Notebook setup
%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle')
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

# Colors setup
project_colors = pplot.colors()
rainbow_cmap = project_colors.colourmap('rainbow')

### Preliminaries

In [ ]:
## Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','RO','VO',
            'ZO']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ro,vo,zo\
    = putil.parse_config_dict(cdict,keywords)

## Pathing
data_dir = base_dir+'data/'
version_dir = 'apogee_'+apogee_dr+'_'+apogee_results_vers+'_gaia_'+gaia_dr+'/'
ga_dir = data_dir+'gaia_apogee/'+version_dir
fig_dir = './fig/'

## Filenames
allstar_filename = ga_dir+'apogee_allstar.npy'
apogee_stat_indx_filename = ga_dir+'apogee_statIndx.npy'
gaia_data_filename = ga_dir+'gaia_data.npy'
gaia_apogee_matches_filename = ga_dir+'gaia_apogee_matches.npy'

### Read data

In [ ]:
# Load APOGEE data
print('APOGEE data release is: '+apogee_dr+', and results version is: '+apogee_results_vers)
print('Loading APOGEE from '+allstar_filename)
allstar = np.load(allstar_filename)
print(str(len(allstar))+' stars in total sample.')

# load APOGEE statistical sample index
print('\nLoading APOGEE DR16 statistical sample from '+apogee_stat_indx_filename)
apogee_stat_indx = np.load(apogee_stat_indx_filename)
print(str(np.sum(apogee_stat_indx))+' stars in statistical sample.')

# Gaia data and Gaia-APOGEE match index
print('\nGaia data release is: '+gaia_dr)
print('Loading Gaia catalog from '+gaia_data_filename)
gaia_data = np.load(gaia_data_filename, allow_pickle=True)
print('Loading Gaia-APOGEE matches from '+gaia_apogee_matches_filename)
gaia_apogee_matches_indx = np.load(gaia_apogee_matches_filename)

# Apply the statistical sample index and Gaia-APOGEE matching index
allstar_input = allstar[apogee_stat_indx][gaia_apogee_matches_indx]

In [ ]:
# Remove bulge fields
omask_bulge = ~(((allstar_input['GLON'] > 340.) |\
                 (allstar_input['GLON'] < 20.)) &\
                (np.fabs(allstar_input['GLAT']) < 20.)
               )
# Remove stars without measured Fe/H
omask_feh = (allstar_input['FE_H'] > -999.)

omask = omask_feh & omask_bulge

gaia = gaia_data[omask]
allstar = allstar_input[omask]

In [ ]:
# Read GC catalogue
gc_tab = pssf.get_harris_catalog(output_filename=None, return_catalog=True)

### Plot

In [ ]:
ap_ll = allstar['GLON']
ap_bb = allstar['GLAT']
gc_ll = gc_tab['GLON']
gc_bb = gc_tab['GLAT']
gc_rh = gc_tab['R_HALF']

plot_ap_ll = copy.deepcopy(ap_ll)
plot_ap_ll[plot_ap_ll>180]-=360
plot_gc_ll = copy.deepcopy(gc_ll)
plot_gc_ll[plot_gc_ll>180]-=360

fig = plt.figure( figsize=(8,6) )
ax = fig.add_subplot(111)

ax.scatter( plot_ap_ll, ap_bb, s=1, c='Black', alpha=0.1 )
ax.scatter( plot_gc_ll, gc_bb, s=10, c='DodgerBlue', alpha=1.0 )

bulge_rect = matplotlib.patches.Rectangle(xy=(-20,-20), width=40, height=40, 
                                          facecolor='None', 
                                          edgecolor='Red')
ax.add_patch(bulge_rect)

ax.set_xlabel(r'$\ell$', fontsize=15)
ax.set_ylabel(r'$b$', fontsize=15)
ax.set_xlim(-180,180)
ax.set_ylim(-90,90)
ax.invert_xaxis()
fig.set_facecolor('White')
plt.show()

### Determine which fields are contaminated

In [ ]:
# Make skycoords
gc_skycoord = SkyCoord(l=gc_ll*apu.deg, b=gc_bb*apu.deg, frame=Galactic)
apogee_skycoord = SkyCoord(l=ap_ll*apu.deg, b=ap_bb*apu.deg, frame=Galactic)

In [ ]:
# Find the nearest GC to each APOGEE star
idx, d2d, _ = apogee_skycoord.match_to_catalog_sky(gc_skycoord)

In [ ]:
# Get the tidal radius of the GCs matched to each star
nearest_gc_rh = gc_rh[idx]
nearest_gc_rh[ ~np.isfinite(nearest_gc_rh) ] = 0 # Ignore if no measured r_h, probably really distant GC
# Find stars with a GC nearby, within 3 tidal radii
where_gc_near_star = np.where( d2d < (12*nearest_gc_rh.data*apu.arcmin) )[0]
# List of fields that contain a GC nearby
fields_with_gc = allstar['LOCATION_ID'][where_gc_near_star]
unique_fields_with_gc = np.unique(fields_with_gc)
unique_gc_indices = np.unique(idx[where_gc_near_star])

In [ ]:
print('Offending fields:')
print('Total N: '+str(len(unique_fields_with_gc)))
print(unique_fields_with_gc)

In [ ]:
print('Offending globular clusters:')
print('Total N: '+str(len(unique_gc_indices)))
print(unique_gc_indices)
print(gc_tab['ID'][unique_gc_indices])

In [ ]:
for i in range( len(unique_fields_with_gc) ):
    
    print('-------------------------------------------------------------'
          '-----------------------------------------')
    
    # Show the stars from this field, base the plotting properties on those 
    # within 3 tidal radii of the overlapping GC
    field_locid = unique_fields_with_gc[i] 
    where_this_locid = np.where( allstar['LOCATION_ID'] == field_locid )[0]
    # Mask for stars that are actually close to the GC, based on 3 tidal radii
    rt_mask = d2d[where_this_locid] <\
              (12*nearest_gc_rh[where_this_locid].data*apu.arcmin)
    gc_close = idx[where_gc_near_star[np.where(fields_with_gc ==\
                                               unique_fields_with_gc[i])[0]]]
        
    fig = plt.figure(figsize=(12,6))
    ax1 = fig.add_subplot(231)
    ax2 = fig.add_subplot(232)
    ax3 = fig.add_subplot(233)
    ax4 = fig.add_subplot(234)
    ax5 = fig.add_subplot(235)
    ax6 = fig.add_subplot(236)
    
    # GC properties for colorbar windows
    gc_feh = np.unique(gc_tab['FE_H'][gc_close]).data[0]
    if np.isnan(gc_feh):
        gc_feh = -1.
    feh_vmin = gc_feh - 1.
    feh_vmax = gc_feh + 1.
    
    gc_rv = np.unique(gc_tab['V_HELIO'][gc_close]).data[0]
    if np.isnan(gc_rv):
        gc_rv = 0.
    gc_vsig = np.unique(gc_tab['SIGMA_V'][gc_close]).data[0]
    if np.isnan(gc_vsig):
        gc_vsig = 20
    n_vsig = 2.
    rv_vmin = gc_rv - n_vsig*gc_vsig
    rv_vmax = gc_rv + n_vsig*gc_vsig
    
    # Spatial coloring by [Fe/H] and histogram
    pts_1 = ax1.scatter(ap_ll[where_this_locid], ap_bb[where_this_locid], 
                        s=4, cmap=rainbow_cmap, vmin=feh_vmin, vmax=feh_vmax, 
                        alpha=1.0, c=allstar['FE_H'][where_this_locid], 
                        )
    cbar1 = fig.colorbar( pts_1, ax=ax1 )
    cbar1.ax.set_ylabel('[Fe/H]')
    tr_circle1 = plt.Circle((gc_ll[gc_close].data[0], gc_bb[gc_close].data[0]), 
                            gc_rh[gc_close].data[0]*12/60, 
                            facecolor='None', edgecolor='Black', 
                            linestyle='dashed')
    ax1.add_artist(tr_circle1)
    ax1.set_xlabel(r'$\ell$', fontsize=15)
    ax1.set_ylabel(r'$b$', fontsize=15)
    
    ax4.hist(allstar['FE_H'][where_this_locid][~rt_mask], histtype='bar', 
             facecolor='None', edgecolor='Gray', range=[feh_vmin,feh_vmax],
             linewidth=6., zorder=1, alpha=0.75)
    ax4.hist(allstar['FE_H'][where_this_locid][rt_mask], histtype='bar', 
             facecolor='None', edgecolor='Red', range=[feh_vmin,feh_vmax], 
             linewidth=4., zorder=2)
    ax4.set_xlabel('Fe/H')
    ax4.axvline(np.unique(gc_tab['FE_H'][gc_close]).data[0], 
                linestyle='dashed', color='DodgerBlue', linewidth=2., zorder=3)
    
    # Spatial coloring by [alpha/Fe] and histogram
    pts_2 = ax2.scatter( ap_ll[where_this_locid], ap_bb[where_this_locid], 
                        s=4, cmap=rainbow_cmap, vmin=0, vmax=0.4, 
                        c=allstar['MG_FE'][where_this_locid], alpha=1.0 )
    cbar2 = fig.colorbar( pts_2, ax=ax2 )
    cbar2.ax.set_ylabel('[Mg/Fe]')
    tr_circle2 = plt.Circle((gc_ll[gc_close].data[0], gc_bb[gc_close].data[0]), 
                            gc_rh[gc_close].data[0]*12/60, 
                            facecolor='None', edgecolor='Black', 
                            linestyle='dashed')
    ax2.add_artist(tr_circle2)
    ax2.set_xlabel(r'$\ell$', fontsize=15)
    ax2.set_ylabel(r'$b$', fontsize=15)
    
    ax5.hist(allstar['MG_FE'][where_this_locid][~rt_mask], histtype='bar', 
             facecolor='None', edgecolor='Gray', range=[0.,0.4],
             linewidth=6., zorder=1, alpha=0.75)
    ax5.hist(allstar['MG_FE'][where_this_locid][rt_mask], histtype='bar', 
             facecolor='None', edgecolor='Red', range=[0.,0.4],
             linewidth=4., zorder=2)
    ax5.set_xlabel('[Mg/Fe]')
    
    # Spatial coloring by V_HELIO and histogram
    pts_3 = ax3.scatter(ap_ll[where_this_locid], ap_bb[where_this_locid], 
                        s=4, cmap=rainbow_cmap, vmin=rv_vmin, vmax=rv_vmax, 
                        alpha=1.0, c=allstar['VHELIO_AVG'][where_this_locid], 
                        )
    cbar3 = fig.colorbar( pts_3, ax=ax3 )
    cbar3.ax.set_ylabel(r'$V_\mathrm{HELIO}$')
    tr_circle3 = plt.Circle((gc_ll[gc_close].data[0], gc_bb[gc_close].data[0]), 
                            gc_rh[gc_close].data[0]*12/60, 
                            facecolor='None', edgecolor='Black', 
                            linestyle='dashed')
    ax3.add_artist(tr_circle3)
    ax3.set_xlabel(r'$\ell$', fontsize=15)
    ax3.set_ylabel(r'$b$', fontsize=15)
    
    ax6.hist(allstar['VHELIO_AVG'][where_this_locid][~rt_mask], histtype='bar', 
             facecolor='None', edgecolor='Gray', linewidth=6., zorder=1,
             range=[gc_rv - 2*n_vsig*gc_vsig, gc_rv + 2*n_vsig*gc_vsig], 
             alpha=0.75)
    ax6.hist(allstar['VHELIO_AVG'][where_this_locid][rt_mask], histtype='bar', 
             facecolor='None', edgecolor='Red', linewidth=4., zorder=2,
             range=[gc_rv - 2*n_vsig*gc_vsig, gc_rv + 2*n_vsig*gc_vsig] )
    ax6.set_xlabel(r'$V_\mathrm{HELIO}$')
    ax6.axvline(np.unique(gc_tab['V_HELIO'][gc_close]).data[0], 
                linestyle='dashed', color='DodgerBlue', linewidth=2., zorder=3)
    

    
    
    # ax5.scatter(allstar['GLON'][where_this_locid], 
    #             allstar['GLAT'][where_this_locid] )
    # ax5.axhline(20,  linestyle='solid', color='Blue')
    # ax5.axhline(-20, linestyle='solid', color='Blue')
    # ax5.axvline(340, linestyle='solid', color='Blue')
    # ax5.axvline(20,  linestyle='solid', color='Blue')
        
    # ax5.set_xlabel('Galactic Longitude')
    # ax5.set_ylabel('Galactic Latitude')
    # ax5.set_xlim(0,360)
    # ax5.set_ylim(-90,90)
    
    fig.set_facecolor('White')
    plt.tight_layout()
    
    print('Field name: '+str(field_locid))
    print('\nNearby GC:')
    print( str('ID: ')+str(np.unique(gc_tab['ID'][gc_close]).data[0]) )
    print( str('Name: ')+str(np.unique(gc_tab['NAME'][gc_close]).data[0]) )
    print( str('[Fe/H]: ')+str(np.unique(gc_tab['FE_H'][gc_close]).data[0]) )
    print( str('(l,b): (')+str(np.unique(gc_ll[gc_close]).data[0])+\
          ', '+str(np.unique(gc_bb[gc_close]).data[0])+')' )
    print('\nProperties of stars')
    
    feh_m1_mask = allstar['FE_H'][where_this_locid] < -1
    feh_m06_mask = allstar['FE_H'][where_this_locid] < -0.6
    feh_m1_3rt_mask = allstar['FE_H'][where_this_locid][rt_mask] < -1
    feh_m06_3rt_mask = allstar['FE_H'][where_this_locid][rt_mask] < -0.6
    print('Number of stars at [Fe/H] < -1: '+str(np.sum(feh_m1_mask)) )
    print('Number of stars in 3 tidal radii at [Fe/H] < -1: '+str(np.sum(feh_m1_3rt_mask)) )
    print('Number of stars at [Fe/H] < -0.6: '+str(np.sum(feh_m06_mask)) )
    print('Number of stars in 3 tidal radii at [Fe/H] < -0.6: '+str(np.sum(feh_m06_3rt_mask)) )
    
    plt.show()

    print('------------------------------------------------------------------------------------------------------')
###i